In [1]:
from pymongo import MongoClient
import pprint 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time
import copy 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [14]:
epl2020 = 'https://fbref.com/en/comps/9/3232/2019-2020-Premier-League-Stats' #Assigning the url
r = requests.get(epl2020)
r.status_code #Making sure we get the all good code
soup = BeautifulSoup(r.text, 'html.parser') #Pass into beautiful soup
tables = soup.find_all('table') #Grabing all the tables on the website
indices = tables[0].find_all('th') #Grabing the the table we want
rows = tables[0].find_all('tr')
columns = {}
for index in indices: #checking out the data
    columns[index.text] = None
all_data = []
keys = list(columns.keys())
for i,row in enumerate(rows): #Getting my data ready to put into pandas
    if i > 0:
        new_row = copy.copy(columns)
        entries = row.find_all('td')
        for j,entry in enumerate(entries):
            new_row[keys[j+1]]= entry.text
        all_data.append(new_row)
for dic in range(20): #Adjusting my data
    all_data[dic]['Rk'] = (dic+1)
df = pd.DataFrame(all_data) #Making a pandas df
dlst = list(range(17,38)) #Cleaning pandas df
df.drop(df.columns[dlst], axis = 1, inplace = True) 
df = df.style.hide_index()  
df 

Rk,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Attendance,Top Team Scorer,Goalkeeper
1,Liverpool,38,32,3,3,85,33,+52,99,71.5,40.0,+31.5,+0.83,"41,955",Mohamed Salah - 19,Alisson
2,Manchester City,38,26,3,9,102,35,+67,81,93.0,34.7,+58.3,+1.53,"37,097",Raheem Sterling - 20,Ederson
3,Manchester Utd,38,18,12,8,66,36,+30,66,59.4,37.4,+22.0,+0.58,"57,415","Marcus Rashford, Anthony Martial - 17",David de Gea
4,Chelsea,38,20,6,12,69,54,+15,66,66.6,37.9,+28.6,+0.75,"32,023",Tammy Abraham - 15,Kepa Arrizabalaga
5,Leicester City,38,18,8,12,67,41,+26,62,61.6,44.5,+17.1,+0.45,"25,312",Jamie Vardy - 23,Kasper Schmeichel
6,Tottenham,38,16,11,11,61,47,+14,59,46.1,52.0,-6.0,-0.16,"43,757",Harry Kane - 18,Hugo Lloris
7,Wolves,38,15,14,9,51,40,+11,59,47.1,34.8,+12.3,+0.32,"24,758",Raúl Jiménez - 17,Rui Patrício
8,Arsenal,38,14,14,10,56,48,+8,56,49.2,56.6,-7.3,-0.19,"47,589",Pierre-Emerick Aubameyang - 22,Bernd Leno
9,Sheffield Utd,38,14,12,12,39,39,0,54,41.5,47.9,-6.4,-0.17,"24,370","Oliver McBurnie, Lys Mousset - 6",Dean Henderson
10,Burnley,38,15,9,14,43,50,-7,54,43.9,48.3,-4.4,-0.12,"15,995",Chris Wood - 14,Nick Pope


In [10]:
keys

['Rk',
 'Squad',
 'MP',
 'W',
 'D',
 'L',
 'GF',
 'GA',
 'GDiff',
 'Pts',
 'xG',
 'xGA',
 'xGDiff',
 'xGDiff/90',
 'Attendance',
 'Top Team Scorer',
 'Goalkeeper',
 'Notes',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20']